In [ ]:
# MOCK TESTS WITH PYTEST
# 

from unittest import mock

import pytest
from typing import TYPE_CHECKING, Literal, Optional, Union
from src.DAO.movie_dao import MovieDAO
from datetime import date
from src.Model.movie import Movie
from tests.conftest import avatar  # Assurez-vous que `avatar` est bien défini ici

class MockMovieDBConnector:
    def __init__(self, movie=None):
        self.movie = movie or avatar  # Utilise l'avatar si aucun film n'est fourni

    def sql_query(
        self,
        query: str,
        data: Optional[Union[tuple, list, dict]] = None,
        return_type: Union[Literal["one"], Literal["all"]] = "one",
    ):
        match query:
            case "SELECT * FROM movie WHERE id_movie=%s":
                if not data:
                    raise Exception("No data provided.")
                id_movie = data[0]  # Récupère l'ID à partir de la donnée
                if id_movie == 19995:
                    return Movie(**self.movie)
        return None  # Pour d'autres requêtes non définies

@pytest.fixture
def mock_db_connection():
    return MockMovieDBConnector()

def test_get_movie_by_id(mock_db_connection):
    movie_dao = MovieDAO(mock_db_connection)
    movie = movie_dao.get_by_id(19995)
    
    assert movie is not None
    assert movie.id_movie == 19995
    assert movie.title == "Avatar"
    assert movie.adult is False
    assert movie.budget == 237000000

def test_get_movie_by_non_existent_id(mock_db_connection):
    movie_dao = MovieDAO(mock_db_connection)
    movie = movie_dao.get_by_id(99999)  # ID qui n'existe pas
    
    assert movie is None  # Vérifiez que la méthode renvoie None pour un film inexistant



In [3]:
from src.Service.password_service import hash_password, create_salt

password = "NoCode"
salt = ["tes", "tuser", "2"]
hashed_password = hash_password(password)
print(hashed_password)    

9da90a168c5026a3291f7612de2caebc7e36ae4c630416fc3332a41a1c630fee


In [ ]:
from src.Model.connected_user import ConnectedUser
from src.Model.genre import Genre
from src.Model.movie import Movie
from datetime import date
d = Movie(
                id_movie=1,
                title="Movie 1",
                belongs_to_collection=None,
                budget=1000000,
                genres=[Genre(id=123, name="Musical Comedy")],
                origin_country=["US"],
                original_language="en",
                original_title="Movie 1",
                overview="Overview 1",
                popularity=8.5,
                release_date=date(2023, 1, 1),
                revenue=5000000,
                runtime=120,
                vote_average=7.5,
                vote_count=1000,
                adult=False,
            )
print (d)

Title : Movie 1, ( ID: 1 ), Release Date: 2023-01-01, Popularity: 8.5, Vote Average: 7.5, Vote Count: 1000


In [ ]:
result= [
            {
                "id_user": 2,
                "username": "user2",
                "hashed_password": "hashed2",
                "date_of_birth": date(1992, 2, 2),
                "gender": 2,
                "first_name": "First2",
                "last_name": "Last2",
                "email_address": "user2@example.com",
                "password_token": "token2",
                "phone_number": "0987654321",
                "own_film_collection": [2],
                "follow_list": [1],
            }
        ]
user = dict(result)
print(ConnectedUser(**user))

ValueError: dictionary update sequence element #0 has length 12; 2 is required